# LangChain と Gemini で RAG を作ってみる

### RAG
1. あらかじめ用意したテキスト(群)を数値化(ベクター化/embedding)し、ベクトルDBに保存
2. テキストに対する質問文を数値化し、数値化されたベクトルDB中の各文書との距離(数値的差)を求め、距離の小さいものの上位を候補として抽出
3. 抽出されたテキストを、質問文とともにLLMへ投げると、質問文に合わせてテキストを解釈し、回答

pip install google-genai  
pip install langchain langchain-community langchain-google-genai  
pip install unstructured[pdf,html]  
pip install chromadb  
pip install python-magic-bin # C library for windows  

In [1]:
import os

import google.genai as genai
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

In [2]:
# Gemini API Keyを取得
with open('GOOGLE_API_KEY.txt', 'r') as f:  # ファイルからAPI Keyを取得
    api_key = f.read().strip()

# Geminiモデルを指定
llm_model       = 'gemini-2.0-flash-exp'
enbedding_model = 'gemini-embedding-001'

## 簡単なLangchane+ChramaDB+Geminiを利用した、RAGの例

In [3]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

texts = [
    '石川です。エンジニアです。散歩は嫌いではありません。', 
    '果物屋の山田です。以前は八百屋でした。甘いものが好きです。下戸です。',
    '佐藤さんは酒屋を営んでいました。現在はコンビニの店長です。泣き上戸です。',
    '東京の渡辺さんと佐藤さんは、たまに居酒屋で一緒に飲んでいるようです。',
    'サイクリングの好きな鈴木さんは自転車で会社へ通勤しています。高橋さんは会社の同僚です。',
    '伊藤さんと山本さんは、よく一緒に奥多摩へキャンプに行くようです',
    '中村さんは高橋さんの勤めている会社の上司です。よく飲みに誘われますが参加するかは半々です。',
    '京都にお住いの小林さんは、佐藤さんのおいです。',
    '加藤さんは5人家族です。',
]
# Document化する
docs = [Document(page_content=t, metadata={}) for t in texts]

# テキストをエンベディング(vector化)する関数
embeddings = GoogleGenerativeAIEmbeddings(model=enbedding_model, google_api_key=api_key)

# VectorStoreを初期化する
try:
    vectorstore.delete_collection()
except (NameError, AttributeError):
    pass
# VectorStoreとしてChroma DBをメモリ上に作成し、Documentを格納 (永続化しない)
vectorstore = Chroma.from_documents(
    docs,          # 格納対象のDocument
    embeddings     # エンベディング関数
)

# 確認: VectorStore -------------------------
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})  # 検索関数, k件取得
query = '山田さんの職業は何ですか。'
docs = retriever.invoke(query)    # クエリを投げて検索結果を取得
# 確認: retriever結果
print(f"test vectorstore: Question={query}")
for i, doc in enumerate(docs, start=1):
    print(f"--- retrieved doc {i} ---" + 
          f"\nmetadata={doc.metadata}\ncontent={doc.page_content[:200]}") # 最初の200文字だけ表示
# -------------------------------------------

# 作成したVectorStoreからの検索関数を設定
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})  # k件取得

# Gemini LLMの設定 (RAGの生成部分)
llm = ChatGoogleGenerativeAI(
    model=llm_model,           # モデル名
    google_api_key=api_key,    # API Key
    temperature=1.0            # Default=0.7
)

# 検索と生成を統合したQ&Aチェーンの作成
qa_chain = RetrievalQA.from_chain_type(
    retriever=retriever,
    chain_type='stuff',         # 取得したドキュメントをプロンプトへ追加
    llm=llm,
    return_source_documents=True 
)

queries = []
queries.append('佐藤さんの職業は何ですか。')
queries.append('お酒を飲む人は誰ですか。')
queries.append('関東に住んでいる人は誰ですか。')
print(f"\nQ&A ----------------")
for i, query in enumerate(queries):
    result = qa_chain.invoke(query)          # 質問実行
    answer   = result['result']              # 回答部分
    evidence = result['source_documents']    # 入力ドキュメントを表示
    print(f"\nQ{i} {query}; answer={answer}")
    for i, e in enumerate(evidence, start=1):
        print(f"evidence{i}={e.page_content}")

test vectorstore: Question=山田さんの職業は何ですか。
--- retrieved doc 1 ---
metadata={}
content=果物屋の山田です。以前は八百屋でした。甘いものが好きです。下戸です。
--- retrieved doc 2 ---
metadata={}
content=サイクリングの好きな鈴木さんは自転車で会社へ通勤しています。高橋さんは会社の同僚です。

Q&A ----------------

Q0 佐藤さんの職業は何ですか。; answer=佐藤さんは以前は酒屋を経営していましたが、現在はコンビニの店長です。
evidence1=佐藤さんは酒屋を営んでいました。現在はコンビニの店長です。泣き上戸です。
evidence2=京都にお住いの小林さんは、佐藤さんのおいです。
evidence3=東京の渡辺さんと佐藤さんは、たまに居酒屋で一緒に飲んでいるようです。

Q1 お酒を飲む人は誰ですか。; answer=文脈から、お酒を飲むのは渡辺さんと佐藤さん、そして高橋さんと中村さんであることがわかります。
evidence1=東京の渡辺さんと佐藤さんは、たまに居酒屋で一緒に飲んでいるようです。
evidence2=果物屋の山田です。以前は八百屋でした。甘いものが好きです。下戸です。
evidence3=中村さんは高橋さんの勤めている会社の上司です。よく飲みに誘われますが参加するかは半々です。

Q2 関東に住んでいる人は誰ですか。; answer=渡辺さんが東京に住んでいます。
evidence1=京都にお住いの小林さんは、佐藤さんのおいです。
evidence2=東京の渡辺さんと佐藤さんは、たまに居酒屋で一緒に飲んでいるようです。
evidence3=加藤さんは5人家族です。


## 大きなドキュメントの分割、メタデータの活用(予定)

ドキュメントとして、青空文庫を利用させていただきました。

(メタデータはDocumentへの付与まで)

In [4]:
import os
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

novels_dir = './novels/'
# テキストファイル・メタデータ
matadata_dic = {
    '1_kumono_ito.txt': {
        'genre': 'novel', 'title':'蜘蛛の糸',          'author':'芥川竜之介', 'year':'1918'},
    '2_serohikino_goshu.txt': {
        'genre': 'novel', 'title':'セロ弾きのゴーシュ', 'author':'宮沢賢治',   'year':'1934'},
    '3_bocchan.txt': {
        'genre': 'novel', 'title':'坊ちゃん',         'author':'夏目漱石',   'year':'1906'},
    'x_wagahaiwa_nekodearu.txt': {
        'genre': 'novel', 'title':'吾輩は猫である',    'author':'夏目漱石',   'year':'1905'},
    '4_kaijin_nijumenso.txt': {
        'genre': 'novel', 'title':'怪人二十面相',      'author':'江戸川乱歩', 'year':'1936'},
}
null_dic = {'genre':'','title':'', 'author':'','year':''}

# 小説データをロード
loader = DirectoryLoader(novels_dir, glob='*.txt')
documents = loader.load()   # ドキュメント(メタデータ+テキスト)としてロード

# 各ドキュメントにメタデータを付与
for doc in documents:
    fn = os.path.basename(doc.metadata["source"])
    doc.metadata.update({'filename': fn} | matadata_dic.get(fn, null_dic))
    print(doc.metadata) # 確認

# ドキュメントをチャンクへ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=100,
    separators=['。', '\n'] # 分割位置の指定
)
chunks = text_splitter.split_documents(documents)

# テキストをエンベディング(vector化)する関数
embeddings = GoogleGenerativeAIEmbeddings(model=enbedding_model, google_api_key=api_key)

# VectorStoreを初期化する
try:
    vectorstore.delete_collection()
except (NameError, AttributeError):
    pass
# VectorStoreとしてChroma DBをメモリ上に作成 (永続化しない)
vectorstore = Chroma.from_documents(chunks, embeddings)
print(f"\nChroma DBをメモリ上に作成しました。")

{'source': 'novels\\1_kumono_ito.txt', 'filename': '1_kumono_ito.txt', 'genre': 'novel', 'title': '蜘蛛の糸', 'author': '芥川竜之介', 'year': '1918'}
{'source': 'novels\\2_serohikino_goshu.txt', 'filename': '2_serohikino_goshu.txt', 'genre': 'novel', 'title': 'セロ弾きのゴーシュ', 'author': '宮沢賢治', 'year': '1934'}
{'source': 'novels\\3_bocchan.txt', 'filename': '3_bocchan.txt', 'genre': 'novel', 'title': '坊ちゃん', 'author': '夏目漱石', 'year': '1906'}
{'source': 'novels\\4_kaijin_nijumenso.txt', 'filename': '4_kaijin_nijumenso.txt', 'genre': 'novel', 'title': '怪人二十面相', 'author': '江戸川乱歩', 'year': '1936'}

Chroma DBをメモリ上に作成しました。


In [5]:
# Retriever 作成
retriever = vectorstore.as_retriever(search_kwargs={'k': 1})  # k件取得

# クエリを投げて結果を取得
query = '御釈迦様はどこにいましたか'
docs = retriever.invoke(query)

# 確認: 検索結果
print(f"test vectorstore: Question={query}")
for i, doc in enumerate(docs, start=1):
    print(f"--- retrieved doc {i} ---" + 
          f"\nmetadata={doc.metadata}\ncontent={doc.page_content[:200]}") # 最初の200文字だけ表示

test vectorstore: Question=御釈迦様はどこにいましたか
--- retrieved doc 1 ---
metadata={'filename': '1_kumono_ito.txt', 'year': '1918', 'title': '蜘蛛の糸', 'author': '芥川竜之介', 'genre': 'novel', 'source': 'novels\\1_kumono_ito.txt'}
content=蜘蛛の糸

芥川龍之介

［＃８字下げ］一［＃「一」は中見出し］

ある日の事でございます。御釈迦様《おしゃかさま》は極楽の蓮池《はすいけ》のふちを、独りでぶらぶら御歩きになっていらっしゃいました。池の中に咲いている蓮《はす》の花は、みんな玉のようにまっ白で、そのまん中にある金色《きんいろ》の蕊《ずい》からは、何とも云えない好《よ》い匂《におい》が、絶間《たえま》なくあたりへ溢《あふ》れて居ります


In [6]:
# 作成したVectorStoreからの検索を設定
retriever = vectorstore.as_retriever(
    search_kwargs={'k':3}      # 返すchank数
)

# Gemini LLMの設定 (RAGの生成部分)
llm = ChatGoogleGenerativeAI(
    model=llm_model,           # モデル名
    google_api_key=api_key,    # API Key
    temperature=1.0
)

# 検索と生成を統合したチェーンの作成
qa_chain = RetrievalQA.from_chain_type(
    retriever=retriever,
    chain_type='stuff',         # 取得したドキュメントをプロンプトへ追加
    llm=llm,
    return_source_documents=True 
)

In [7]:
queries = []
queries.append('蜘蛛の糸は何に使われましたか')
queries.append('ゴーシュは何をしていますか')
queries.append('坊ちゃんの姓名は何ですか')
queries.append('怪人の名前は何ですか')
queries.append('二十面相が現れた場所をすべて挙げてください')  # <= k=3では無理
queries.append('猫の名は何ですか')
for i, query in enumerate(queries):
    result = qa_chain.invoke(query)                       # 質問実行
    answer   = result['result']                           # 回答部分
    # 確認: 検索ドキュメントの表示
    evidence = result['source_documents'] # 入力ドキュメントを表示
    print(f"-----\nQ{i}: {query}, answer={answer},")
    for i, e in enumerate(evidence):
        print(f"\nevidence{i}={e.page_content[:200]}")    # 初めの200文字

-----
Q0: 蜘蛛の糸は何に使われましたか, answer=蜘蛛の糸は、御釈迦様が地獄にいるカンダタを救い出すために垂らされました。,

evidence0=蜘蛛の糸

芥川龍之介

［＃８字下げ］一［＃「一」は中見出し］

ある日の事でございます。御釈迦様《おしゃかさま》は極楽の蓮池《はすいけ》のふちを、独りでぶらぶら御歩きになっていらっしゃいました。池の中に咲いている蓮《はす》の花は、みんな玉のようにまっ白で、そのまん中にある金色《きんいろ》の蕊《ずい》からは、何とも云えない好《よ》い匂《におい》が、絶間《たえま》なくあたりへ溢《あふ》れて居ります

evidence1=。その村はずれの森の中に、みょうなお城のようないかめしいやしきが建っているのです。

まわりには高い土塀をきずき、土塀の上には、ずっと先のするどくとがった鉄棒を、まるで針の山みたいに植えつけ、土塀の内がわには、四メートル幅ほどのみぞが、ぐるっととりまいていて、青々とした水が流れています。深さも背がたたぬほど深いのです。これはみな人をよせつけぬための用心です。たとい針の山の土塀を乗りこえても、その中

evidence2=。十分もすれば、ピッポちゃんは、明智先生のおばさんのところへとんでいくだろう。おばさんはぼくの手紙を読んで、さぞびっくりなさるだろうなあ。でも、すぐに警視庁へ電話をかけてくださるにちがいない。それから警官がここへかけつけるまで、三十分かな？　四十分かな？　なんにしても、今から一時間のうちには、賊がつかまるんだ。そしてぼくは、この穴ぐらから出ることができるんだ。」

小林少年は、ピッポちゃんの消えて
-----
Q1: ゴーシュは何をしていますか, answer=ゴーシュは町の活動写真館でセロを弾く係です。,

evidence0=セロ弾きのゴーシュ

宮沢賢治

ゴーシュは町の活動写真館でセロを弾く係りでした。けれどもあんまり上手でないという評判でした。上手でないどころではなく実は仲間の楽手のなかではいちばん下手でしたから、いつでも楽長にいじめられるのでした。

ひるすぎみんなは楽屋に円くならんで今度の町の音楽会へ出す第六｜交響曲《こうきょうきょく》の練習をしていました。

トランペットは一生けん命歌っています。

ヴァイ

evidence1=。

野ねずみはばたばたしな